In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
import datetime
import warnings
warnings.filterwarnings('ignore')

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from math import *
from sklearn.model_selection import train_test_split

x_centre = 3760901.5068
y_centre = -19238905.6133

train = pd.read_csv('data_train.csv')
test = pd.read_csv('data_test.csv')
train = train.drop(['Unnamed: 0'],axis=1)
test = test.drop(['Unnamed: 0'],axis=1)

tr = test.loc[test.x_exit.isnull()]

def conv(time_1):   
    sec1 = 3600*int(time_1[0:2]) + 60*int(time_1[3:5]) + int(time_1[6:8])                                                                                                                                
    return (sec1)

def distance(x1,x2,y1,y2):
    dist = np.sqrt(((x2-x1)**2)+((y2-y1)**2))
    return dist

def ecartTemps(t1,t2):
    ecart= t2-t1
    return ecart

def distanceCentre(x,y):
    dx = max(0,abs(x_centre-x)-10000)
    dy = max(0,abs(y_centre-y)-30000)
    return (dx**2+dy**2)

ecartTemps = np.vectorize(ecartTemps)

distance = np.vectorize(distance)
distanceCentre = np.vectorize(distanceCentre)
#vitesses négatives

test.vmin[test.vmin < 0] = np.NaN
test.vmean[test.vmean < 0] = np.NaN
test.vmax[test.vmax < 0] = np.NaN

train.vmin[train.vmin < 0] = np.NaN
train.vmean[train.vmean < 0] = np.NaN
train.vmax[train.vmax < 0] = np.NaN

#incohérences
train['vmean'][(train.time_entry==train.time_exit)] = 0
train['vmin'][(train.time_entry==train.time_exit)] = 0
train['vmax'][(train.time_entry==train.time_exit)] = 0

test['vmean'][(test.time_entry==test.time_exit)] = 0
test['vmin'][(test.time_entry==test.time_exit)] = 0
test['vmax'][(test.time_entry==test.time_exit)] = 0

train['vmax'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0
train['vmin'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0
train['vmean'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0

test['vmax'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0
test['vmin'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0
test['vmean'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0

#outliers
train.vmean = train.vmean.loc[(train.vmean<90)]
train.vmax = train.vmean.loc[(train.vmax<90)]
train.vmin = train.vmean.loc[(train.vmin<90)]


#nan by mean
train['vmax']=train['vmax'].replace( np.NaN , train['vmax'].mean() )
train['vmin']=train['vmin'].replace( np.NaN , train['vmin'].mean() )
train['vmean']=train['vmean'].replace( np.NaN , train['vmean'].mean() )

test['vmax']=test['vmax'].replace( np.NaN , test['vmax'].mean() )
test['vmin']=test['vmin'].replace( np.NaN , test['vmin'].mean() )
test['vmean']=test['vmean'].replace( np.NaN , test['vmean'].mean() )

# trainTempVmean = train.loc[(train.vmean>0)]
# trainTempVmean = trainTempVmean.dropna()
# trainTempVmean = trainTempVmean.vmean.mean()

# trainTempVmax = train.loc[(train.vmax>0)]
# trainTempVmax = trainTempVmax.dropna()
# trainTempVmax = trainTempVmax.vmax.mean()

# trainTempVmin = train.loc[(train.vmin>0)]
# trainTempVmin = trainTempVmin.dropna()
# trainTempVmin = trainTempVmin.vmin.mean()

# testTempVmean = test.loc[(test.vmean>0)]
# testTempVmean = testTempVmean.dropna()
# testTempVmean = testTempVmean.vmean.mean()

# testTempVmax = test.loc[(test.vmax>0)]
# testTempVmax = testTempVmax.dropna()
# testTempVmax = testTempVmax.vmax.mean()

# testTempVmin = test.loc[(test.vmin>0)]
# testTempVmin = testTempVmin.dropna()
# testTempVmin = testTempVmin.vmin.mean()


# train['vmax']=train['vmax'].replace( np.NaN , trainTempVmax)
# train['vmin']=train['vmin'].replace( np.NaN , trainTempVmin)
# train['vmean']=train['vmean'].replace( np.NaN , trainTempVmean)
# test['vmax']=test['vmax'].replace( np.NaN , testTempVmax)
# test['vmin']=test['vmin'].replace( np.NaN , testTempVmin)
# test['vmean']=test['vmean'].replace( np.NaN , testTempVmean)

vec_conv =  np.vectorize(conv)
train.time_entry=vec_conv(train.time_entry)
train.time_exit=vec_conv(train.time_exit)

test.time_entry=vec_conv(test.time_entry)
test.time_exit=vec_conv(test.time_exit)

train['duree']=ecartTemps(train.time_entry,train.time_exit)
test['duree']=ecartTemps(test.time_entry,test.time_exit)
train = train.drop(['time_entry','time_exit'],axis=1)
test = test.drop(['time_entry','time_exit'],axis=1)

# train['center']=distance(x_centre,train.x_entry,y_centre,train.y_entry)
# test['center']=distance(x_centre,test.x_entry,y_centre,test.y_entry)

train['distCenter']=distanceCentre(train.x_entry,train.y_entry)
test['distCenter']=distanceCentre(test.x_entry,test.y_entry)


# #(3750901.5068+3770901.5068)/2 x centre
# # = 3760901.5068
# #(-19268905.6133+-19208905.6133)/2
# # = -19238905.6133
# Center = distance(train.x_entry,3760901.5068,train.y_entry,-19238905.6133)
# train['Center'] = Center

train['city_center'] = 0
train['city_center'][(train.x_exit>=3750901.5068) & (train.x_exit<=3770901.5068)&(train.y_exit>=(-19268905.6133)) & (train.y_exit<=(-19208905.6133))]=1


In [ ]:
# Y=train
# Y = Y.groupby('hash')['city_center'].apply(list)
# for i in range (Y.shape[0]):
#     Y[i]=Y[i][-1]

# Y=pd.DataFrame(Y)
# Y = pd.DataFrame(Y.values)

#Y.to_csv("Y.csv",index=False)

train = train.drop(['city_center'],axis=1)
train = train.drop(['trajectory_id'],axis=1)
test = test.drop(['trajectory_id'],axis=1)


def transformOnLine(df):
    lesdataframes=[]

    for vincent in df.columns[1:]:
        print(vincent)
        k=df
        k = k.groupby('hash')[vincent].apply(list)

        if (vincent == 'x_exit') | (vincent == 'y_exit'):
            for i in range (k.shape[0]):
                k[i].reverse()
                k[i]=k[i][1:]      #x_exit y_exit
        else :
            for i in range (k.shape[0]):
                k[i].reverse()


        k=pd.DataFrame(k)


        # expand df.time_entry into its own dataframe
        gouz = k[vincent].apply(pd.Series)

        # rename each variable is time_entry
        gouz = gouz.rename(columns = lambda x : 'tag_' + str(x))
        
        if(vincent == 'distCenter'):
            gouz = gouz.fillna(math.inf)
        else:
            gouz = gouz.fillna(0)
        
        lesdataframes.append(gouz)

    r = pd.concat(lesdataframes,axis=1)
    r = r.reset_index()
    r = r.drop(['hash'],axis=1)
    r.columns = np.arange(len(r.columns))
    return r

In [ ]:
# X = transformOnLine(train)
# X_test = transformOnLine(test)
# X.to_csv("X.csv",index=False)
# X_test.to_csv("X_test.csv",index=False)
X =pd.read_csv('X.csv')
X_test = pd.read_csv('X_test.csv')
Y = pd.read_csv('Y.csv')

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X1, Y, test_size=0.25, random_state=1)
# clf = XGBClassifier()
# clf.fit(X_train,y_train)
# pred = clf.predict(X_test)
# print('f1_score',f1_score(y_test.values.astype(int),pred.astype(int)))
# #0.8783700809377617 baseline
# #0.8795344106678721 distance au centre
# #0.8818456969904631 distance au rectangle

In [ ]:
# centre
# 20000
# sur 60000
# #calculs des angles

# #X[14] X[15] x et y entry
# #X[16] X[17] x et y exit
# #(3750901.5068+3770901.5068)/2 x centre
# # = 3760901.5068
# #(-19268905.6133+-19208905.6133)/2
# # = -19238905.6133

# u = np.array([X[16],X[17]])-np.array([X[14],X[15]])
# v = np.array([3760901.5068-X[14],-19238905.6133-X[15]])

# angle_train = []
# for i in range(u.shape[1]):
#     scalar = (np.dot([u[0][i],u[1][i]],[v[0][i],v[1][i]]))
#     u_norm = (np.linalg.norm([u[0][i],u[1][i]]))
#     v_norm = (np.linalg.norm([v[0][i],v[1][i]]))
#     angle_train.append(degrees(acos(scalar/(u_norm*v_norm))))
    
# u = np.array([X_test[16],X_test[17]])-np.array([X_test[14],X_test[15]])
# v = np.array([3760901.5068-X_test[14],-19238905.6133-X_test[15]])

# angle_test = []
# for i in range(u.shape[1]):
#     scalar = (np.dot([u[0][i],u[1][i]],[v[0][i],v[1][i]]))
#     u_norm = (np.linalg.norm([u[0][i],u[1][i]]))
#     v_norm = (np.linalg.norm([v[0][i],v[1][i]]))
#     angle_test.append(degrees(acos(scalar/(u_norm*v_norm))))

# #application

# X['angle']=angle_train
# X_test['angle']=angle_test

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# #GRID SEARCH

# print(datetime.datetime.now())
# X = X
# y = Y.astype(int)
# param_grid = {
#     'num_boost_round': [60,70,90,150,300],
#     'eta': [0.1,0.15,0.2],
#     'max_depth': [8,10,12,15,20],
#     'min_child_weight': [0.4,0.5],
#     'n_jobs':[-1],
#     'colsample_bytree':[0.3,0.5,0.8],
#     'subsample' :[0.8,1],
#     'gamma' :[0,1,5]
# }
# XGBClassifier()

# kf = StratifiedKFold(n_splits=3,random_state=1,shuffle=True)
# random_search = RandomizedSearchCV(XGBClassifier(),param_grid,n_iter=7, cv=kf.split(X, y),verbose=3, scoring= 'f1',iid=True,n_jobs=-1)
# random_search.fit(X,y)

# print(datetime.datetime.now())


In [ ]:
#pd.DataFrame(random_search.cv_results_)

In [ ]:
import time

print(datetime.datetime.now())
X = X
y = Y.astype(int)
param_grid = {
    'n_jobs':[-1],
    'n_estimators': [70,90,110],
    'eta': [0.01,0.05,0.1],
    'max_depth': [14,15,16],
    'min_child_weight': [0.4,0.5],
    'n_jobs':[-1],
    'colsample_bytree':[0.3,0.5],
    'subsample' :[1],
    'gamma' :[5]
}

kf = StratifiedKFold(n_splits=10,random_state=1,shuffle=True)
model = GridSearchCV(XGBClassifier(),param_grid, cv=kf.split(X, y),verbose=3, scoring= 'f1',iid=True,n_jobs=-1)
model.fit(X,y)

print (model.best_score_
print (model.best_params_)
print(pd.DataFrame(model.cv_results_))

print(datetime.datetime.now())


In [ ]:
model.best_params_

In [ ]:
#model = pickle.load(open('V2_XGB_best.pickle', 'rb'))
pickle.dump(model.best_estimator_,open('XGB_best_5','wb'))

In [ ]:
pred = model.best_estimator_.predict(X_test)
prediction = pd.DataFrame()

In [ ]:
trajectory_id = tr['trajectory_id']

In [ ]:
trajectory_id=trajectory_id.values
prediction['id']=trajectory_id
prediction['target']=pred

In [ ]:
prediction.target = prediction.target.astype(int)
prediction.describe()

In [ ]:
prediction.to_csv("OL_XGB_good3.csv",index=False)

In [ ]:
prediction.target.value_counts()

In [ ]:
# clf = XGBClassifier()
# clf.fit(X,Y)

In [ ]:
# pred = clf.predict(X_test)
# prediction = pd.DataFrame()

In [ ]:
# from matplotlib import pyplot
# pyplot.bar(range(len(clf.feature_importances_[np.argsort(clf.feature_importances_)[-10:]])), clf.feature_importances_[np.argsort(clf.feature_importances_)[-10:]])
# pyplot.show()
# np.argsort(clf.feature_importances_)[-10:]

In [ ]:
# import tensorflow as tf
# import theano # utilise à la fois le Cpu et le Gpu de l'ordinateur
# import keras
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X1=scaler.fit_transform(X)
# X1_test=scaler.transform(X_test)

In [ ]:
# def Build_Classif ():
#     classifier = Sequential()
#     classifier.add(Dense(units=89,activation="relu",kernel_initializer="uniform",input_dim=178)) #1ere couche
#     classifier.add(Dense(units=89,activation="relu",kernel_initializer="uniform")) #couche cachée
#     classifier.add(Dense(units=1,activation="sigmoid",kernel_initializer="uniform")) #to get a proba
#     classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])
#     return classifier

In [ ]:
# classifier = KerasClassifier(build_fn=Build_Classif,batch_size=, epochs=30)